In [9]:
import os
import datetime
import enum
import time
import dataclasses
import struct
import math
import binascii
import typing

import dotenv
import pymongo
import trio
import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh as bk
import numpy as np
import smbus
import RPi.GPIO as GPIO

bk.io.output_notebook()
%autoawait trio

Loading BokehJS ...

______________

## Connect to Database

In [17]:
def connect_to_db():
    """Open the connection to the DB and return the collection
    Create collection with unique index, if there is not yet one"""
    # Load environment variables from .env file
    
    dotenv.load_dotenv()
    
    # Get MongoDB-URI
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["IoT-Project"]

    return db["Raw-Data"]

In [18]:
def represent_for_mongodb(obj):
    match obj:
        case dict():
            return {represent_for_mongodb(k):represent_for_mongodb(v) for k,v in obj.items()}
        case tuple() | list():
            return type(obj)(represent_for_mongodb(v) for v in obj)
        case np.generic():
            return obj.item()
        case _:
            return obj

In [19]:
def insert_data_to_db(data):
    collection = connect_to_db()
    collection.insert_one(
        represent_for_mongodb(data)
    )

____________

## Classes for the Multi-Gas-Sensors

In [20]:
class CmdCode(enum.Enum):
    read_concentration = 0x86
    read_temp = 0x87
    read_all = 0x88

In [21]:
class SensorType(enum.Enum):
    O2         =  0x05
    CO         =  0x04
    H2S        =  0x03
    NO2        =  0x2C
    O3         =  0x2A
    CL2        =  0x31
    NH3        =  0x02
    H2         =  0x06
    HCL        =  0X2E
    SO2        =  0X2B
    HF         =  0x33
    PH3        =  0x45

In [22]:
@dataclasses.dataclass
class SensorData:
    gas_concentration: float  # ppm
    sensor_type: SensorType
    temperature: float        # degree Celsius

In [23]:
class MultiGasSensor:
    """
    Class for all Gas-Sensors
    Adapted from https://github.com/DFRobot/DFRobot_MultiGasSensor
    """
    
    def __init__(self, bus_number: int, i2c_address: int, expected_sensor_type: SensorType | None = None):
        self.i2c_bus = smbus.SMBus(bus_number)
        self.i2c_address = i2c_address
        self.expected_sensor_type = expected_sensor_type

    
    @classmethod
    def calc_check_sum(cls, data:bytes)->int:
        return (~sum(data)+1) & 0xff

    
    def command(self, code: CmdCode, *args:bytes) -> bytes:
        data = bytes([0xFF, 0x01, code.value]) + b"".join(args)
        data += b"\x00"*(8-len(data))
        data += bytes([self.calc_check_sum(data[1:-1])])
        self.i2c_bus.write_i2c_block_data(self.i2c_address, 0, list(data))
        time.sleep(0.1)                                                         # TO-DO: async-sleept machen
        
        result = self.i2c_bus.read_i2c_block_data(self.i2c_address, 0, 9)
        result = bytes(result)
        result_str = binascii.hexlify(result," ",1)
        
        assert result[0] == 0xFF, result_str
        assert result[1] == code.value, result_str
        assert result[8] == self.calc_check_sum(result[1:-2]), f"CRC failure: received 0x{result[8]:02x}, calculated 0x{self.calc_check_sum(result[1:-1]):02x}, ({result_str})"

        return result[2:-1]

    
    def read_all(self) -> SensorData:
        result = self.command(CmdCode.read_all)

        # '>': big-endian encoded struct (MSB first: most significant byte first);
        # 'H': 2 Bytes unsigned integer ("half long integer")
        # 'B': 1 Byte unsigned integer  ("byte")
        gas_concentration_raw, sensor_type, decimal_places, temperature_raw = struct.unpack(">HBBH", result)
        gas_concentration = gas_concentration_raw * 10**-decimal_places
        sensor_type = SensorType(sensor_type)
   
        Vpd3 = 3*temperature_raw/1024 # Spannung in Volt
        Rth = Vpd3*10000/(3-Vpd3) # Spannung mit Spannnungsteiler vonem 10k-Widerstand
        temperature = 1/(1/(273.15+25)+1/3380.13*(math.log(Rth/10000)))-273.15 # Transfer-Kurve von temperaturfühler mit 10kOhm bei 25°C und alpha-Wert von 3380.13

        if self.expected_sensor_type is not None:
            assert sensor_type == self.expected_sensor_type
        
        return SensorData(
            gas_concentration=gas_concentration,
            sensor_type=sensor_type,
            temperature=temperature,
        )

__________

## Handle Alert-Mode

In [24]:
async def wait_for_alert_end(max_wait_time, previous_button_state) -> tuple[typing.Literal["timeout", "button pressed", "no more alert"], bool]:
    
    with trio.move_on_after(max_wait_time): # Runs until max_wait_time is over
        while True:
            current_button_state = GPIO.input(switch)==GPIO.LOW
            if not previous_button_state and current_button_state:
                return "button pressed", previous_button_state

            if not (o2_alert):
                return "no more alert", previous_button_state

            previous_button_state = current_button_state
            await trio.sleep(0.05)
    
    return "timeout", previous_button_state

In [25]:
async def alert():
    """
    Check if an alert is present or not
    If alert is not present: normal mode and 10ms sleep
    If alert is present: LED & Buzzer on and blinking
    """   
    while True:  

        """NORMAL-MODE"""
        if not (o2_alert):
            normal_mode()
            await trio.sleep(0.1)
            continue

        
        """ALERT-MODE"""
        GPIO.output(led_green, GPIO.LOW)
        button_state = True
        
        while True:

            # Blink-On-Phase
            GPIO.output(o2_sensor, GPIO.HIGH if o2_alert else GPIO.LOW)
            GPIO.output(buzzerpin, GPIO.HIGH)
            
            result, button_state = await wait_for_alert_end(0.4, button_state)           
            if result != "timeout":
                break # -> to ACKNOWLEDGE-MODE or to Normal-State
            
            # Blink-Off-Phase
            GPIO.output(o2_sensor, GPIO.LOW)
            GPIO.output(buzzerpin, GPIO.LOW)
            
            result, button_state = await wait_for_alert_end(0.4, button_state)
            if result != "timeout":
                break # -> to ACKNOWLEDGE-MODE or to Normal-State

        
        if result == "button pressed":
            """
            ACKNOWLEDGE-MODE:
            - LEDs on, Buzzer off
            - If Button is pressed again: ACKNOWLEDGE-MODE is aborted
            -> alert-mode is reactivated if there is still an alert, otherwise it goes back to normal
            """
            previous_button_state = True # Button was pressed before
            GPIO.output(buzzerpin, GPIO.LOW)

            while o2_alert:
                current_button_state = GPIO.input(switch)==GPIO.LOW

                if (not previous_button_state and current_button_state): # Checks if there was a change from not-pressed to pressed
                    break
                
                GPIO.output(o2_sensor, GPIO.HIGH if o2_alert else GPIO.LOW)

                previous_button_state = current_button_state
                await trio.sleep(0.1)

In [26]:
def normal_mode():
    """
    Mode without alert
    """    
    GPIO.output(led_green, GPIO.HIGH)
    GPIO.output(o2_sensor, GPIO.LOW)
    GPIO.output(buzzerpin, GPIO.LOW)

___________

## Create Plot

In [27]:
def fig_setup(o2_alert_level):
    time = np.array([], dtype=np.datetime64)
    
    o2_danger_zone = bokeh.models.BoxAnnotation(top=o2_alert_level, fill_alpha=0.2, fill_color='#D55E00')
    o2_safe_zone = bokeh.models.BoxAnnotation(bottom=o2_alert_level, fill_alpha=0.2, fill_color='#0072B2')   

    ds = bk.models.ColumnDataSource(data=dict(time=time, oxygenlevel=[])) # like empty df

    
    # O2-Plot
    o2_fig = bk.plotting.figure(x_axis_type="datetime", height=400, width=800)
    o2_fig.line(x="time",y="oxygenlevel",source=ds, legend_label="O2-Level")
    o2_fig.add_layout(o2_danger_zone)
    o2_fig.add_layout(o2_safe_zone)
    o2_fig.title.text="Oxygen"
    
    p = bokeh.layouts.gridplot([[o2_fig]])
    handle = bokeh.io.show(p, notebook_handle=True) # um Daten nachzuschieben im Notebook

    return handle, ds

In [28]:
def plot(handle, ds, time, oxygen):
    ds.stream(dict(time=[time], oxygenlevel=[oxygen]), rollover=600) # Neue Daten werden an das Ende der Kolonnen angehängt, ab 600 werden die ältesten Daten herausgeworfen
    bk.io.push_notebook(handle=handle)

_________

## Aggregate Data

In [29]:
def aggregate_data(alldata):
    data = np.array([data for time, data in alldata])

    min = data.min()
    max = data.max()
    avg = data.mean()

    aggregation = dict(
        min=min,
        max=max,
        avg=avg,
    )
    
    return aggregation

In [30]:
def send_data_to_mqqt(type, short, data):
    string = f"{type}:\t{data}"
    publish.single(f'sensor/{short}', string, hostname='127.0.0.1')

_________

## Measurement-Loop

In [31]:
async def measurement(*,measurement_interval=0.1, aggregation_interval=10):
    
    global o2_alert

    o2_alert_level = 20.5 # Normally below 17%

    
    handle, ds = fig_setup(o2_alert_level)
    next_measurement = trio.current_time()+measurement_interval
    next_aggregation = trio.current_time()+aggregation_interval

    i=0
    while True:
        alldata_o2 = []
        
        while trio.current_time()<next_aggregation:      
            time = datetime.datetime.now().astimezone(None).astimezone(datetime.timezone.utc)
            if True:
                try:
                    oxygen = O2.read_all().gas_concentration
                    print(oxygen)
                except Exception as ex:
                    #print(f'{ex!r} - retry')
                    await trio.sleep(0.05)
                    continue
            
            else:
                data = {}
                for k, sensor in dict(o2=O2).items():
                    try:
                        result = sensor.read_all()
                    except Exception as ex:
                        # print(f"{k} sensor failed: {ex!r}")
                        continue
                    data[k] = result.gas_concentration
                if not data:
                    # none of the sensors worked - retry soon
                    await trio.sleep(0.05)
                    continue
                        
            alldata_o2.append((time, oxygen))
            
            plot(handle, ds, time, oxygen)
            
            # Check if alerts are True
            aggregation_o2 = aggregate_data(alldata_o2)
            o2_alert = oxygen<o2_alert_level

            await trio.sleep_until(next_measurement)
            next_measurement += measurement_interval

        # Aggregate Data
        next_aggregation += aggregation_interval

        aggregation = dict(
            time=time,
            O2=aggregation_o2,
        )

        insert_data_to_db(aggregation)
        
        print(aggregation)  

________

## Setup

In [67]:
i2cbus          = 1
O2_ADDRESS      = 0x77

In [68]:
o2_alert        = False

In [69]:
O2              = MultiGasSensor(i2cbus, O2_ADDRESS, SensorType.O2)

In [70]:
GPIO.setmode(GPIO.BOARD) # use BOARD PIN Numbering

#o2_sensor      = 19
#o2_sensor       = 23
#o2_sensor       = 32
o2_sensor       = 21

led_green       = 31
buzzerpin       = 38
switch          = 29

In [71]:
def setup():
    # O2-LED
    GPIO.setup(o2_sensor, GPIO.OUT) # set the red ledPin to OUTPUT mode
    GPIO.output(o2_sensor, GPIO.LOW) # make red ledPin output LOW level
    
    # Green LED
    GPIO.setup(led_green, GPIO.OUT)
    GPIO.output(led_green, GPIO.HIGH) # make green ledPin output HIGH level

    # Buzzer-Pin
    GPIO.setup(buzzerpin, GPIO.OUT) # set buzzerPin to OUTPUT mode

    # Button
    GPIO.setup(switch, GPIO.IN, pull_up_down=GPIO.PUD_UP) # set switch to PULL UP INPUT mode

______________

## Run Script

In [72]:
setup()

try: 
    async with trio.open_nursery() as nursery:
        nursery.start_soon(alert)
        nursery.start_soon(measurement)
finally:
    GPIO.cleanup()

KeyboardInterrupt: 